In [1]:
pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 27.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-many

In [3]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM
import torch
import torch.nn as nn
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModelForCausalLM
from transformers import BertConfig, BertForSequenceClassification
import pandas as pd
import random
from transformers import AutoTokenizer, AutoModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertModel
from safetensors import safe_open
import os
from transformers import BertTokenizer

#모델과 토크나이저 로드
config_poly = PeftConfig.from_pretrained("Sakong/polyglot-ko-5.8-1000")
base_model = AutoModelForCausalLM.from_pretrained("EleutherAI/polyglot-ko-5.8b")
model_poly = PeftModel.from_pretrained(base_model, "Sakong/polyglot-ko-5.8-1000")
tokenizer_poly = AutoTokenizer.from_pretrained("Sakong/polyglot-ko-5.8-1000")

Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/299 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
# CustomBERTModel 클래스 정의 (필요 시 추가)
class CustomBERTModel(nn.Module):
    def __init__(self, model_name, num_emotion_labels, num_empathy_labels):
        super(CustomBERTModel, self).__init__()
        self.bert = BertModel.from_pretrained(model_name)
        self.emotion_classifier = nn.Linear(self.bert.config.hidden_size, num_emotion_labels)
        self.empathy_classifier = nn.Linear(self.bert.config.hidden_size, num_empathy_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]
        emotion_logits = self.emotion_classifier(pooled_output)
        empathy_logits = self.empathy_classifier(pooled_output)
        return emotion_logits, empathy_logits

# 모델 및 토크나이저 로드
model_path = '/content/drive/MyDrive/nlp_FinalProject/InteractiveEmpathyTrainingSystem/finetuned_kobert'
model_kobert = CustomBERTModel('monologg/kobert', num_emotion_labels=6, num_empathy_labels=4)

# 각 구성 요소의 가중치 로드
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
with safe_open(os.path.join(model_path, 'model.safetensors'), framework="pt") as f:
    state_dict = {k: f.get_tensor(k) for k in f.keys()}
model_kobert.bert.load_state_dict(state_dict)
model_kobert.emotion_classifier.load_state_dict(torch.load(os.path.join(model_path, 'emotion_classifier.pth'), map_location=device))
model_kobert.empathy_classifier.load_state_dict(torch.load(os.path.join(model_path, 'empathy_classifier.pth'), map_location=device))

model_kobert.eval()

tokenizer_kobert = BertTokenizer.from_pretrained(model_path)

config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [43]:
# 프로그램 실행 시 출력되는 문장 선택
def get_random_sentence(csv_file):
    df = pd.read_csv(csv_file)
    return random.choice(df['utterance_text'].tolist())


# 감정/공감 분석
def analyze_emotion(text, model, tokenizer):
    inputs = tokenizer_kobert(text, return_tensors='pt', max_length=256, truncation=True, padding='max_length')
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    with torch.no_grad():
        emotion_logits, empathy_logits = model_kobert(input_ids, attention_mask)
        emotion_label = torch.argmax(emotion_logits, dim=1).item()
        empathy_label = torch.argmax(empathy_logits, dim=1).item()
    return emotion_label, empathy_label

label_map_emotion = {0: "기쁨", 1: "당황", 2: "분노", 3: "불안", 4: "상처", 5: "슬픔"}
label_map_empathy = {0: "동조", 1: "격려", 2: "조언", 3: "위로"}



# 시스템의 답변 생성
def generate_response(role_message, text):
    # 입력을 처리하여 토크나이저로 변환
    prompt = f"시스템: {text}\n상대방: {role_message}\n너: "
    inputs = tokenizer_poly(prompt, return_tensors="pt", padding=True, truncation=True, max_length=512)
    # `generate` 함수를 호출할 때, `max_length`와 `no_repeat_ngram_size` 파라미터를 사용
    outputs = model_poly.generate(
        inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=100,  # 최대 길이 설정
        no_repeat_ngram_size=2,  # 같은 n-gram이 반복되지 않도록 설정
        pad_token_id=tokenizer_poly.pad_token_id,  # 패딩 토큰 설정
        num_return_sequences=1
    )
    response = tokenizer_poly.decode(outputs[0], skip_special_tokens=True)
    response = response.replace('"', '')  # 응답에서 따옴표 제거

    # '너: ' 이후의 내용만 추출
    if '너: ' in response:
        response = response.split('너: ')[1].strip()
    # 프롬프트에 포함된 내용 제거
    response = response.replace(text, '').replace(role_message, '').strip()
    return response


# 코사인 유사도 계산
def cosine_similarity_sentences(sentence1, sentence2):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([sentence1, sentence2])
    cosine_sim = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])
    return cosine_sim[0][0]



def main():
    print("공감형 대화 학습 시스템이 시작되었습니다. '나가기'를 입력하면 프로그램이 종료됩니다.")
    user_input = input("""
                  1. 부모자녀,조손
                  2. 연인
                  3. 지인
                  4. 친구
                  5. 형제,자매
                  6. 랜덤
                  대화할 상대를 선택해 정수로 입력하세요 : """)
    role_message = ""
    try:
        if user_input == "1":
            selected_csv = "/content/drive/MyDrive/nlp_FinalProject/InteractiveEmpathyTrainingSystem/random_sentence/1.csv"
            role_message = "너는 부모/조부모 역할이고, 나는 자녀,손자 역할이야. 이 역할에 맞게 답변해줘."
        elif user_input == "2":
            selected_csv = "/content/drive/MyDrive/nlp_FinalProject/InteractiveEmpathyTrainingSystem/random_sentence/2.csv"
            role_message = "너는 나와 연인 관계야. 이 역할에 맞게 답변해줘."
        elif user_input == "3":
            selected_csv = "/content/drive/MyDrive/nlp_FinalProject/InteractiveEmpathyTrainingSystem/random_sentence/3.csv"
            role_message = "너는 나와 지인 관계야. 이 역할에 맞게 답변해줘."
        elif user_input == "4":
            selected_csv = "/content/drive/MyDrive/nlp_FinalProject/InteractiveEmpathyTrainingSystem/random_sentence/4.csv"
            role_message = "너는 나와 친구 관계야. 이 역할에 맞게 답변해줘."
        elif user_input == "5":
            selected_csv = "/content/drive/MyDrive/nlp_FinalProject/InteractiveEmpathyTrainingSystem/random_sentence/5.csv"
            role_message = "너는 나와 형제자매 관계야. 이 역할에 맞게 답변해줘."
        elif user_input == "6":
            selected_csv = "/content/drive/MyDrive/nlp_FinalProject/InteractiveEmpathyTrainingSystem/random_sentence/6.csv"  # 랜덤 선택 시 별도의 역할 부여 없음
        else:
            print("잘못된 입력입니다. 1부터 6까지의 숫자를 입력해주세요.")
    except ValueError as e:
      print(e)
      return

    sentence = get_random_sentence(selected_csv)
    print("시스템: ", sentence)
    model_response = generate_response(role_message, sentence)


    while True:
      #print("모델 응답: ", model_response) # 개발 중 확인용 출력, 나중에 제거 예정
      user_response = input("대답: ")
      if user_response.lower() == "나가기":
        print("프로그램 종료.")
        break

      sim_score = cosine_similarity_sentences(model_response, user_response)
      if sim_score >= 0.4:
        print("----대화를 계속 진행합니다.")
        model_response = generate_response(role_message, user_response)  # 사용자의 답변에 이어서 모델이 응답
        print("모델 응답: ", model_response)
      else:
        while True:
                emotion_label, empathy_label = analyze_emotion(sentence, model_kobert, tokenizer_kobert)
                predicted_emotion, predicted_empathy = label_map_emotion[emotion_label], label_map_empathy[empathy_label]
                print(f"""<피드백>
            방금은 {predicted_emotion}을/를 드러내는 상황이었어요.
            {predicted_empathy}을/를 나타내는 말을 해보세요.
                """)
                print("시스템: ", sentence)  # 사용자에게 다시 동일한 문장 제시
                user_response = input("대답: ")
                user_emotion_label, user_empathy_label = analyze_emotion(user_response, model_kobert, tokenizer_kobert)
                if user_empathy_label == predicted_empathy:
                    print('아주 잘했어요. 적절한 답변이에요.')
                    model_response = generate_response(role_message, user_response)
                    print("모델 응답: ", model_response)
                    break  # 내부 루프를 탈출하고, 외부 루프는 계속 실행
                '''else:
                    print("올바른 반응을 다시 시도해주세요.")
                    continue  # 같은 피드백 루프를 반복'''



if __name__ == "__main__":
    main()

공감형 대화 학습 시스템이 시작되었습니다. '나가기'를 입력하면 프로그램이 종료됩니다.

                  1. 부모자녀,조손
                  2. 연인
                  3. 지인
                  4. 친구
                  5. 형제,자매
                  6. 랜덤
                  대화할 상대를 선택해 정수로 입력하세요 : 2
시스템:  나 오늘도 가족들이랑 크게 싸웠어. 사소한 문제였는데 감정 싸움으로 번지더라고.
대답: 가족들이랑 사소한 문제로 싸우다니.. 감정 소모가 컸겠다. 마음이 아프겠다
<피드백>
            방금은 기쁨을/를 드러내는 상황이었어요. 
            위로을/를 나타내는 말을 해보세요.
                
시스템:  나 오늘도 가족들이랑 크게 싸웠어. 사소한 문제였는데 감정 싸움으로 번지더라고.
대답: ㅠㅠ 안타깝네 정말.. 왜 싸웠어?
<피드백>
            방금은 기쁨을/를 드러내는 상황이었어요. 
            위로을/를 나타내는 말을 해보세요.
                
시스템:  나 오늘도 가족들이랑 크게 싸웠어. 사소한 문제였는데 감정 싸움으로 번지더라고.
대답: 감정싸움 그거 정말 힘들지,, 다음부터는 싸우지 않으면 좋겠다
<피드백>
            방금은 기쁨을/를 드러내는 상황이었어요. 
            위로을/를 나타내는 말을 해보세요.
                
시스템:  나 오늘도 가족들이랑 크게 싸웠어. 사소한 문제였는데 감정 싸움으로 번지더라고.


KeyboardInterrupt: Interrupted by user